In [1]:
%reload_ext autoreload
%autoreload 2

import time
from tqdm import tqdm
import numpy as np
import scipy as sp
import torch
import torch.nn.functional as F
from torch import nn
from torchvision.transforms import v2 as transforms

from matplotlib import pyplot as plt

from networkAlignmentAnalysis.models.registry import get_model
from networkAlignmentAnalysis import utils
from networkAlignmentAnalysis import datasets
from networkAlignmentAnalysis import files

DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
print('using device: ', DEVICE)

using device:  cuda


In [ ]:
# TODO
# 1.1. include additional AlignmentModel methods stored in extra class in base model
# -- the measure_eigenfeature method is baaaaad -- because it doesn't account for convolutional layers

# I should also break apart the measure_eigenfeatures method for better parameter handling
# and also to allow use of components of the method in other contexts

# 2. Make CIFAR class
# 4. Rewrite existing analysis pipelines
# 5. SLURM!!!!


# Analyses:
# - targeted dropout throughout training curves (blue and black in alignmentMNIST.ipynb)
# - compare initial to final alignment...
# - compare initial alignment to delta weight norm...
# - observe alignment of weight norm
# - compare alignment to outgoing weight norm!


# alignmentShaping.ipynb has an adversarial experiment worth looking at

In [78]:
from pathlib import Path

base_path = Path(r'C:\Users\Andrew\Documents\machineLearning\results\alignment_comparison\lr\MLP\MNIST\SGD')
res_path = base_path / 'resdir'
torch_path = base_path / 'torchdir'
small_path = base_path / 'smalldir'
diff_path = base_path / 'diffdir'
if not res_path.exists(): res_path.mkdir()
if not torch_path.exists(): torch_path.mkdir()
if not small_path.exists(): small_path.mkdir()
if not diff_path.exists(): diff_path.mkdir()

In [43]:
def save_each(results, path):
    for key, val in results.items():
        np.save(path / f"{key}.npy", val)

def load_each(path):
    files = path.rglob('*.*')
    results = {}
    for f in files:
        results[f.stem] = np.load(f, allow_pickle=True).item()
        #print(f.stem, f.suffix, type(results[f.stem]))
    return results

def save_each_torch(results, path):
    for key, val in results.items():
        torch.save(val, path / f"{key}.pth")

def load_each_torch(path):
    files = path.rglob('*.*')
    results = {}
    for f in files:
        results[f.stem] = torch.load(f)
        #print(f.stem, f.suffix, type(results[f.stem]))
    return results
        
# load results dictionary
t = time.time()
results = np.load(base_path / 'results.npy', allow_pickle=True).item()
print('initial load time:', time.time() - t)

# save and load each file in results using np.save / np.load
t = time.time()
save_each(results, res_path)
print('np save time:', time.time() - t)
t = time.time()
res = load_each(res_path)
print('np load time:', time.time() - t)

# save and load each file in results using torch save / load
t = time.time()
save_each_torch(results, torch_path)
print('torch save time:', time.time() - t)
t = time.time()
restorch = load_each_torch(torch_path)
print('torch load time:', time.time() - t)

initial load time: 181.21236991882324
np save time: 151.89708423614502
np load time: 193.31731343269348
torch save time: 150.4975814819336
torch load time: 156.98896884918213


In [52]:
restorch['train_results'].keys()

dict_keys(['loss', 'accuracy', 'alignment', 'delta_weights', 'avgcorr'])

In [51]:
from copy import deepcopy
res_small = deepcopy(restorch)

In [90]:
for key in res_small['train_results']:
    for i in range(30):
        res_small['train_results'][key][i] = res_small['train_results'][key][i][:300]

In [87]:
res_diff = deepcopy(res_small)

In [88]:
to_reshape = ['alignment', 'delta_weights', 'avgcorr']
for key in to_reshape:
    res_diff['train_results'][key] = utils.condense_values(res_diff['train_results'][key])
    print([rd.shape for rd in res_diff['train_results'][key]])

[torch.Size([30, 300, 100]), torch.Size([30, 300, 100]), torch.Size([30, 300, 50]), torch.Size([30, 300, 10])]
[torch.Size([30, 300, 100]), torch.Size([30, 300, 100]), torch.Size([30, 300, 50]), torch.Size([30, 300, 10])]
[torch.Size([30, 300, 1]), torch.Size([30, 300, 1]), torch.Size([30, 300, 1]), torch.Size([30, 300, 1])]


In [92]:
# save and load each file in results using torch save / load
t = time.time()
save_each_torch(res_small, small_path)
print('torch save time:', time.time() - t)
t = time.time()
_ = load_each_torch(small_path)
print('torch load time:', time.time() - t)

torch save time: 7.339666128158569
torch load time: 7.758648872375488


In [103]:
# save and load each file in results using torch save / load
t = time.time()
save_each_torch(res_diff, diff_path)
print('torch save time:', time.time() - t)
t = time.time()
_ = load_each_torch(diff_path)
print('torch load time:', time.time() - t)

torch save time: 0.1612858772277832
torch load time: 0.1667790412902832


In [104]:
t = time.time()
torch.save(res_diff, base_path / 'diff_direct.pth')
print('diff save time: ', time.time() - t)


t = time.time()
xxx = torch.load(base_path / 'diff_direct.pth')
print('diff load time: ', time.time() - t)



diff save time:  0.16904735565185547
diff load time:  0.14708733558654785
